<a href="https://colab.research.google.com/github/imendezguerra/wandb_tutorial/blob/main/wandb_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weights and biases (W&B) tutorial
Resources:
- [W&B main page](https://wandb.ai)
- [W&B documentation](https://docs.wandb.ai)
- [W&B courses](https://https://wandb.ai/site/courses/)

Weights and biases (W&B) is a popular MLOps platform used to track and manage machine-learning experiments. It aims to bring order, reproducibility, and efficiency to the chaotic world of ML experimentation.

To do this W&B provides (among other specialised features):
- Experiment tracking
- Model and dataset versioning
- Visualisations
- Hyperparameter sweeps
- Collaborative environment
- Integrations with common AI-tools (PyTorch, TensorFlow, JAX, Keras, HuggingFace, etc.)

Together, these capabilities make ML research, faster, more reliable, reproducible, scalable, and collaborative.

In this tutorial we will cover the basic functions going through:
- W&B registration and installation
- logging, downloading, and using artifacts
- logging models, metadata, and metrics
- performing hyperparameter sweeps
- visualising metrics (W&B website)
- creating a report (W&B website)

The notebook is structured as follows:
1. [W&B registration](#1-wb-registration)
2. [W&B installation](#2-wb-installation)
3. [Dataset preparation](#3-prepare-dataset)
4. [Model training](#4-model-training)
5. [Model optimisation](#5-model-optimisation)
6. [Model evaluation](#6-model-evaluation)


## 1. W&B registration
W&B offers a free forever plan for academic research including:
- coordinating projects remotely
- unlimited tracking hours, teams, projects
- 100GB free cloud storage

Sign-up in [wandb.ai](https://wandb.ai) using your academic email to unlock the benefits.

## 2. W&B installation
To install the `wandb` package run the following commands:

In [1]:
!pip install wandb
import wandb
wandb.login()

  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 6.9 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.4 MB/s  0:00:00 eta 0:00:01
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [wandb]m16/17 [wandb]hon]]re]


wandb: Currently logged in as: im4417 (icl_img) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## 3. Prepare dataset 


## 4. Model training

## 5. Model optimisation

## 6. Model evaluation